In [1]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import load
from sklearn.metrics import classification_report

<Figure size 1000x600 with 0 Axes>

In [2]:
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=420000) #700 * 60 * 10 (10mins)

Using ready-made DataFrame


In [3]:
df.shape

(1770, 20)

In [4]:
y = df['label'].to_numpy()-1
X = df.drop(['label','subject'],axis=1)
X = X.to_numpy()

X = X.reshape(*X.shape, 1)  # 轉回 CNN 需要的格式

In [5]:
# 分割訓練測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# 建立 1D CNN 模型
def build_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(4, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(8, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [7]:
def build_cnn_lstm(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(4, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(8, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.LSTM(16, return_sequences=False))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [8]:
# 初始化模型
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = len(np.unique(y))
model = build_cnn(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 10
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# 評估模型
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc * 100:.2f}%')

Epoch 1/10


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4796 - loss: 2.9587 - val_accuracy: 0.5330 - val_loss: 0.7677
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5646 - loss: 0.6922 - val_accuracy: 0.6422 - val_loss: 0.6238
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6677 - loss: 0.6070 - val_accuracy: 0.6215 - val_loss: 0.6185
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6434 - loss: 0.6004 - val_accuracy: 0.6384 - val_loss: 0.6130
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6673 - loss: 0.5984 - val_accuracy: 0.6422 - val_loss: 0.6081
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6553 - loss: 0.5964 - val_accuracy: 0.6422 - val_loss: 0.6008
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6692 - loss: 0.5891 - val_accuracy: 0.6422 - val_loss: 0.5925
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6946 - loss: 0.5698 - val_accuracy: 0.7119 - val_loss: 0.5845
Epo

In [9]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

           1       0.64      0.65      0.65       266
           2       0.65      0.64      0.64       265

    accuracy                           0.65       531
   macro avg       0.65      0.65      0.65       531
weighted avg       0.65      0.65      0.65       531



In [10]:
# 初始化模型
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = len(np.unique(y))
model = build_cnn_lstm(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 10
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# 評估模型
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc * 100:.2f}%')

Epoch 1/10


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5145 - loss: 0.6929 - val_accuracy: 0.5009 - val_loss: 0.6922
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5486 - loss: 0.6916 - val_accuracy: 0.5932 - val_loss: 0.6886
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5884 - loss: 0.6845 - val_accuracy: 0.7194 - val_loss: 0.6551
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7503 - loss: 0.6220 - val_accuracy: 0.6855 - val_loss: 0.5455
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7676 - loss: 0.4906 - val_accuracy: 0.7495 - val_loss: 0.4710
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7809 - loss: 0.4204 - val_accuracy: 0.7797 - val_loss: 0.4178
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8204 - loss: 0.3658 - val_accuracy: 0.8249 - val_loss: 0.3673
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8136 - loss: 0.3528 - val_accuracy: 0.8550 - val_loss: 0.3383
Epo

In [11]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
              precision    recall  f1-score   support

           1       0.84      0.85      0.85       266
           2       0.85      0.84      0.84       265

    accuracy                           0.85       531
   macro avg       0.85      0.85      0.85       531
weighted avg       0.85      0.85      0.85       531

